In [1]:
import pandas as pd
import pandas_datareader as web
import yfinance as yf

from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

from numpy.linalg import inv, eig, svd
from sklearn.manifold import TSNE
from sklearn.decomposition import KernelPCA

import numpy as np
import matplotlib.pyplot as plt
from pandas import read_csv, set_option
from pandas.plotting import scatter_matrix 
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from matplotlib.ticker import FuncFormatter

In [2]:
# load dataset
#scraping wikipedia to fetch S&P 500 stock list
snp500url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
SP_stocks = pd.read_html(snp500url)[0]
SP_tickers = SP_stocks['Symbol'].to_list()

dji = (
    pd.read_html('https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average')[1]
)
dji_symbols = dji.Symbol.tolist()

In [3]:
def download_stocks(tickers,start='2014-01-01',end='2019-01-01'):
    return yf.download(tickers, start = start, end = end)

def remove_null(df, cutoff = .99):
    df = df.dropna(thresh=int(df.shape[0]*cutoff),axis=1)
    df = df.dropna(thresh=int(df.shape[1]*cutoff))
    return df

def winsorize(df,q=.025):
    clipped = df.clip(lower=df.quantile(q=q),
                   upper=df.quantile(q=1-q),
                   axis=1)
    return clipped

def df_scaler(df):
    scaler = StandardScaler().fit(df)
    return scaler

def train_test_split(df,percentage = .8):
    
    train_size = int(len(df)*percentage)
    train_set = df[:train_size]
    test_set = df[train_size:]
    
    return train_set, test_set

def normalize_data(df_train,df_test):
    scaler = StandardScaler().fit(df_train)
    
    train = pd.DataFrame(scaler.transform(df_train),
                         columns = df_train.columns,
                         index = df_train.index)
    test = pd.DataFrame(scaler.transform(df_test),
                        columns = df_test.columns,
                        index = df_test.index)
    
    return train, test

In [4]:
SP_data = download_stocks(SP_tickers)['Adj Close']
SP_data.index = pd.to_datetime(SP_data.index)

dji_data = download_stocks(dji_symbols)['Adj Close']
dji_data.index = pd.to_datetime(dji_data.index)

SP_data = remove_null(SP_data)
dji_data = remove_null(dji_data)

[*********************100%***********************]  503 of 503 completed

12 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- OGN: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- CARR: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- CTVA: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- GEHC: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- FOX: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- CEG: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- MLM: No data found for this date range, symbol may be delisted
- OTIS: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- FOXA: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- DOW: Data doesn't exist for startDate = 1388552400, endDate = 1546318800
- BRK.B: No timezone found, symbol may be delisted
[*********************100%*****************

In [5]:
#SP_returns = SP_data.pct_change().dropna()
#dji_returns = dji_data.pct_change().dropna()

SP_log_returns = np.log(SP_data/SP_data.shift(1))[1:]
dji_log_returns = np.log(dji_data/dji_data.shift(1))[1:]

SP_log_returns = winsorize(SP_log_returns)
dji_log_returns = winsorize(dji_log_returns)

In [6]:
SP_train, SP_test = train_test_split(SP_log_returns)
dji_train, dji_test = train_test_split(dji_log_returns)

In [10]:
SP_train_scaled = (SP_train-SP_train.mean())/SP_train.std()
SP_test_scaled = (SP_test-SP_train.mean())/SP_train.std()

dji_train_scaled = (dji_train-dji_train.mean())/dji_train.std()
dji_test_scaled = (dji_test-dji_train.mean())/dji_train.std()

In [11]:
SP_train_corr = 1/(SP_train_scaled.shape[0]-1)*SP_train_scaled.T.dot(SP_train_scaled)
dji_train_corr = 1/(dji_train_scaled.shape[0]-1)*dji_train_scaled.T.dot(dji_train_scaled)

In [13]:
PCA_SP = PCA().fit(SP_train_scaled)
PCA_DJI = PCA().fit(dji_train_scaled)

In [14]:
dji_eigvals, dji_eigvecs = np.linalg.eigh(dji_train_corr)

dji_eigvals = dji_eigvals[::-1]
dji_eigvecs = dji_eigvecs[:,::-1]


print(np.allclose(PCA_DJI.explained_variance_ratio_,(dji_eigvals/np.sum(dji_eigvals))))

print(np.allclose(np.abs(np.dot(PCA_DJI.components_,dji_eigvecs)),
            np.eye(dji_eigvecs.shape[0])))

True
True


In [15]:
SP_eigvals, SP_eigvecs = np.linalg.eigh(SP_train_corr)

print(np.allclose(PCA_SP.explained_variance_ratio_[::-1],(SP_eigvals/np.sum(SP_eigvals))))

print(np.allclose(np.abs(np.dot(PCA_SP.components_,SP_eigvecs[::,::-1])),
            np.eye(SP_eigvecs.shape[0])))

True
True


In [16]:
dji_evecs_std = (dji_eigvecs.T/(dji_train.std().values)).T